In [1]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

In [2]:
# start frozen lake env
env = gym.make("FrozenLake-v0")

In [3]:
# create the Q-table
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size,action_space_size))

In [4]:
# let's see q_table
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [70]:
# initializting Q-learning parameters
num_episodes = 10000 # play the game 10000 times
max_steps_per_episode = 100 # every game just play 100 steps. even not win or lose

learning_rate = 0.1 # we just use the information 10% of each time
discount_rate = 0.99 # what we think about the furture

exploration_rate = 1 # at first we should make exploration
max_exploration_rate = 1 # we can't pass over 1 becourse it's not mean anythihng
min_exploration_rate = 0.01 # even we get all the information , we should still exploration
exploration_decay_rate = 0.01 # well ,after 100 times we will just use the min I don't know if it is all right

In [71]:
# The next course prove my concern ,exploration decay rate is too big , so it will become less exploration to fast
exploration_decay_rate = 0.001 # This time will make the 1000 episodes to be the min rate

In [72]:
# recorde all the rewards
rewards_all_episodes = []

In [73]:
# Q-learning algorithm
for episode in range(num_episodes): # which means 10000 times game
    # initialize new episode params
    state = env.reset() # Which is define in the episodic game.Every episode is different from others just restart
    done = False # This is the condition given by the env , if it is always false ,every episode will step 100
    reward_current_episode = 0 # initial this episode's reward . It will given by the env and accurate.It is the return for the agent.
    
    # what happened each step:
    for step in range(max_steps_per_episode): # which means 100 steps
        # Exploration-exploitation trade-off this is easy just tell you how to trade off in math and impliment
        exploration_rate_threhold = random.uniform(0,1)
        if exploration_rate_threhold > exploration_rate:
            action = np.argmax(q_table[state,:])
        else:
            action = env.action_space.sample()
                
        # Take new action thank you for env , I think even alphago will use the same expression. env.step
        new_state,reward,done,info = env.step(action)
        # Update Q-table turns out use is easier to understand it.
        q_table[state,action] = q_table[state,action] * (1 - learning_rate) + learning_rate * (reward + discount_rate*np.max(q_table[new_state,:]))
        # Set new state
        state = new_state
        # Add new reward
        reward_current_episode += reward
        
        # game over buddy
        if done == True:
            break 
        
    # Exploration rate decay which means decay after every episode.It looks like the decay is not like linear decay as I thought
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)
    # Add current episode reward to total rewards list which will see the result
    rewards_all_episodes.append(reward_current_episode)

In [74]:
# results time
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000
print("********Average reword per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count,":",str(sum(r/1000))) # this is mean
    count += 1000

********Average reword per thousand episodes********

1000 : 0.060000000000000046
2000 : 0.18300000000000013
3000 : 0.36800000000000027
4000 : 0.5770000000000004
5000 : 0.6220000000000004
6000 : 0.6850000000000005
7000 : 0.6690000000000005
8000 : 0.6690000000000005
9000 : 0.7060000000000005
10000 : 0.6680000000000005


In [75]:
# Print the updated Q-table
print("\n*****Q-table*****\n")
print(q_table)


*****Q-table*****

[[0.54979911 0.48490656 0.47064936 0.48264138]
 [0.24479668 0.26930774 0.28986074 0.49443765]
 [0.39766536 0.39934401 0.38650105 0.45802324]
 [0.26102448 0.36372579 0.26587155 0.42152691]
 [0.56812497 0.39230691 0.36137309 0.45522783]
 [0.         0.         0.         0.        ]
 [0.20258606 0.17680912 0.2928456  0.20744526]
 [0.         0.         0.         0.        ]
 [0.31978946 0.46644102 0.28845309 0.59436039]
 [0.46261897 0.67938271 0.36608331 0.4364657 ]
 [0.67532147 0.35699839 0.31378095 0.25309971]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.44482214 0.46259286 0.79795219 0.42066588]
 [0.70669799 0.94505402 0.74459707 0.72245988]
 [0.         0.         0.         0.        ]]


In [77]:
# Watch our agent play the game according to the Q-table

# Let's play 3 times:
for episode in range(10):
    # initialize new episode params
    state = env.reset()
    done = False
    print("*******EPISODE {} *********\n\n\n".format(episode+1))
    time.sleep(1)
    # show every step
    for step in range(max_steps_per_episode):
        # Show current state of env on screen
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        # Choose action with highest Q-value for currnet state
        action = np.argmax(q_table[state,:])
        # Take new action
        new_state,reward,done,info = env.step(action)
        # game over
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1: #win
                print("*****You reach the goal!**********")
                time.sleep(3)
            else: # lose
                print("*******You fell though a hole!********************")
                time.sleep(3)
                clear_output(wait=True)
            break
        # Set new state

  (Left)
SFFF
FHFH
FFFH
HFFG
*******You fell though a hole!********************
